In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install plotly_express

## Goal
The goal of this notebook is essentially to recommend a driver's lifetime value (LTV). 

In [0]:
import os

# change to path
PATH='/content/drive/My Drive/Colab Notebooks/lyft'
os.chdir(PATH)

DATA_DIR = os.path.join(PATH, 'data')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

import warnings
warnings.simplefilter(action='ignore')

## Read in

In [0]:
ride_timestamps = pd.read_csv(os.path.join(DATA_DIR, 'ride_timestamps.csv'))
ride_ids = pd.read_csv(os.path.join(DATA_DIR, 'ride_ids.csv'))
driver_ids = pd.read_csv(os.path.join(DATA_DIR, 'driver_ids.csv'))

In [6]:
ride_timestamps.head()

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


In [7]:
ride_ids.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100


In [8]:
driver_ids.head()

,driver_id,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00
2,011e5c5dfc5c2c92501b8b24d47509bc,2016-04-05 00:00:00
3,0152a2f305e71d26cc964f8d4411add9,2016-04-23 00:00:00
4,01674381af7edd264113d4e6ed55ecda,2016-04-29 00:00:00


## EDA

### `drivers_ids`

In [9]:
driver_ids['driver_id'].nunique() # len(driver_ids)

937

In [10]:
driver_ids['driver_onboard_date'].min(), driver_ids['driver_onboard_date'].max()

('2016-03-28 00:00:00', '2016-05-15 00:00:00')

The file `driver_ids.csv` consists of drivers who signed up as a driver with Lyft between 2016/03/28 to 2016/05/15.

### `ride_timestamps`

Check whether do the rides start after all the drivers have registered.

In [0]:
# convert timestamp to datetime object
ride_timestamps['timestamp'] = pd.to_datetime(ride_timestamps['timestamp'])

In [12]:
ride_timestamps['timestamp'].min(), ride_timestamps['timestamp'].max()

(Timestamp('2016-03-28 05:48:18'), Timestamp('2016-06-27 00:50:50'))

The dataset for rides started on 2016/03/28 and ended on 2016/06/27. We proceed to perform feature engineering on the `timestamp` of each ride.

In [0]:
# Feature engineer time features
def add_datepart(df, fldname, drop=False, time=True):
    """
    Taken from fast.ai
    Helper function that adds columns relevant to a date.
    """
    import re 
    
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear']
    if time: attr = attr + ['Hour', 'Minute']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [14]:
add_datepart(ride_timestamps, 'timestamp')

ride_timestamps.head()

,ride_id,event,timestamp,timestampMonth,timestampWeek,timestampDay,timestampDayofweek,timestampDayofyear,timestampHour,timestampMinute,timestampElapsed
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,6.0,24.0,13.0,0.0,165.0,9.0,39.0,1465810759
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,6.0,24.0,13.0,0.0,165.0,9.0,39.0,1465810791
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,6.0,24.0,13.0,0.0,165.0,9.0,44.0,1465811071
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,6.0,24.0,13.0,0.0,165.0,9.0,44.0,1465811073
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,6.0,24.0,13.0,0.0,165.0,10.0,3.0,1465812185


One of the features we deem important later to perform segmenting on drivers is how long does a driver take, on average, to accept a ride. To do so, we have to calculate the time lag between a ride was requested `event == 'requested_at'` to the time the driver accepted the ride `event == 'accepted_at'`.

We hypothesize that keen drivers who are more likely to have a short window between those two events. We call this new feature `pickup_window`.

In [0]:
# calculate time lag wrt to `requested_at`
def seconds_lag(t):
    delta_to_t0 = t - t.iloc[0]
    return delta_to_t0

In [16]:
%%time
ride_timestamps['pickup_window'] = ride_timestamps.groupby('ride_id')['timestamp'].apply(seconds_lag)

ride_timestamps['pickup_window'] = ride_timestamps['pickup_window'].apply(lambda o: o.total_seconds())

CPU times: user 1min 55s, sys: 3.32 s, total: 1min 58s
Wall time: 1min 54s


In [17]:
ride_timestamps.head()

,ride_id,event,timestamp,timestampMonth,timestampWeek,timestampDay,timestampDayofweek,timestampDayofyear,timestampHour,timestampMinute,timestampElapsed,pickup_window
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,6.0,24.0,13.0,0.0,165.0,9.0,39.0,1465810759,0.0
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,6.0,24.0,13.0,0.0,165.0,9.0,39.0,1465810791,32.0
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,6.0,24.0,13.0,0.0,165.0,9.0,44.0,1465811071,312.0
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,6.0,24.0,13.0,0.0,165.0,9.0,44.0,1465811073,314.0
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,6.0,24.0,13.0,0.0,165.0,10.0,3.0,1465812185,1426.0


In [18]:
accepted_filter = ride_timestamps[ride_timestamps['event'] == 'accepted_at']

driver_accepted_filter = pd.merge(accepted_filter, ride_ids,
                                  on='ride_id')

window_g = driver_accepted_filter.groupby('driver_id')['pickup_window'].mean()

window_g = window_g.reset_index()

window_g.head()

,driver_id,pickup_window
0,002be0ffdc997bd5c50703158b7c2491,8.407942
1,007f0389f9c7b03ef97098422f902e62,11.838710
2,011e5c5dfc5c2c92501b8b24d47509bc,7.294118
3,0152a2f305e71d26cc964f8d4411add9,14.198953
4,01674381af7edd264113d4e6ed55ecda,12.528000


In [19]:
len(window_g)

844

In [20]:
ride_ids['driver_id'].nunique()

937

There are 93 drivers in `ride_ids` who do not have records in `ride_timestamps`.

## `ride_ids`
We would like to get an idea of how many rides has a driver completed over the course of the dataset. We call this feature `ride_count`. It will also be a crucial variable to calculate LTV later.

In [21]:
ride_id_g = ride_ids.groupby('driver_id')['ride_id'].count()

ride_id_g = ride_id_g.reset_index()

ride_id_g.rename(columns={'ride_id': 'ride_count'}, inplace=True)

ride_id_g.sort_values(by='ride_count', ascending=False).head()

,driver_id,ride_count
354,5ccc0e6dc9c7475caf785cdce7b8eb7a,919
308,4eb382d1f7d50fae1294964263d1ce82,831
509,844e9be5a30d8d9c1f8e9ddb086ff717,821
401,689bdf87fb2de49f98bf4946cfaa5068,794
217,3788dc9e91f1548816ce8b5af07ddadc,783


In [22]:
driver_ids['driver_id'].nunique(), ride_ids['driver_id'].nunique()

(937, 937)

In [23]:
len(set(driver_ids['driver_id'].unique().tolist()).intersection(set(ride_ids['driver_id'].unique().tolist())))

854

In [24]:
len(driver_ids[~driver_ids['driver_id'].isin(ride_ids['driver_id'].unique())])

83

Interestingly, even we have 937 unique `driver_id`s for both `driver_ids` and `ride_ids`, only 854 overlap. That means, 83 drivers that appear in ride_ids do not have any record in `driver_ids`.

In [28]:
ride_timestamps['ride_id'].nunique(), ride_ids['ride_id'].nunique()

(194081, 193502)

In [29]:
len(set(ride_timestamps['ride_id'].unique().tolist()).intersection(set(ride_ids['ride_id'].unique().tolist())))

184819

Only 184,819 unique `ride_id`s exist in `ride_timestamps` and `ride_ids`.

To calculate the fare, we first convert the `ride_distance` from metres to mile and convert the `ride_duration` from seconds to minutes.

In [0]:
ride_ids['ride_distance'] = ride_ids['ride_distance'] / 1609.34

ride_ids['ride_duration'] = ride_ids['ride_duration'] / 60

To calculate the fare, we employed the assumptions about the Lyft rate card in the prompt. The formula we use is as follows:

(2 + 1.15 * total distance traveled + 0.22 * total time traveled) * (1 + (prime time / 100)) + 1.75

After calculating the fare, we limited the lower bound of the fare to \$5 and the upper of the fare to \$400.

In [26]:
ride_ids['fare'] = (2 + 
                     ride_ids['ride_distance'] * 1.15 + 
                     ride_ids['ride_duration'] * 0.22) * \
                    ((ride_ids['ride_prime_time'] + 100) / 100) \
                    + 1.75 

ride_ids['fare'].describe()

def limit_bound(fare):
    return min(max(5, fare), 400)

ride_ids['fare'] = ride_ids['fare'].apply(limit_bound)

ride_ids['fare'].describe()

count    193502.000000
mean         13.536544
std           9.881417
min           5.000000
25%           8.036033
50%          10.571234
75%          15.115603
max         400.000000
Name: fare, dtype: float64

In [27]:
fare_g = ride_ids.groupby('driver_id')['fare'].sum()

fare_g = fare_g.reset_index()

fare_g.sort_values(by='fare', ascending=False).head()

,driver_id,fare
354,5ccc0e6dc9c7475caf785cdce7b8eb7a,12350.288077
217,3788dc9e91f1548816ce8b5af07ddadc,12304.887351
308,4eb382d1f7d50fae1294964263d1ce82,10781.942344
412,6b65c06851e944351dd285a1eb729499,10708.943770
509,844e9be5a30d8d9c1f8e9ddb086ff717,10613.901682


We then calculated the sum of the fare for each driver. This will be the total income generated by the driver.

The higher a driver's `ride_prime_time`, the more her or she is valuable to Lyft since they help to meet passengers' demand. This could be another important feature for segmenting drivers. Since we are only interested in the mean of the nonzero `ride_prime_time` values and we do not want 0 values to distort the `ride_prime_time` lower, we replace all zeros with `NaN`.

In [30]:
# make the variable NaN to calculate mean
ride_ids['ride_prime_time'].replace(0, np.nan, inplace=True)

ride_prime_time_g = ride_ids.groupby('driver_id')['ride_prime_time'].mean()

ride_prime_time_g = ride_prime_time_g.reset_index()

ride_prime_time_g.sort_values(by='ride_prime_time', ascending=False).head()

,driver_id,ride_prime_time
807,dae249fc394c9bdf02f7d8bb1ff55733,146.875000
676,b2d3f2fb171a12cac427107690c10089,105.000000
420,6cb35e276085548f3f095a85aa63af7b,98.076923
443,7419cd5c573ff9994c0f8ff5d92b4408,91.071429
655,acd7dc6118befb6724aa3752d1cdbea1,90.277778


In [0]:
# change back to original value
ltv['ride_prime_time'].fillna(0, inplace=True)

Merge both `driver_ids` and `driver_timestamps`.

In [31]:
ids_timestamps = pd.merge(ride_ids, ride_timestamps,
                          left_on='ride_id',
                          right_on='ride_id',
                          how='outer')

ids_timestamps = ids_timestamps.sort_values(by=['driver_id', 'timestamp'])

ids_timestamps.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,fare,event,timestamp,timestampMonth,timestampWeek,timestampDay,timestampDayofweek,timestampDayofyear,timestampHour,timestampMinute,timestampElapsed,pickup_window
1235,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,requested_at,2016-03-29 18:46:50,3.0,13.0,29.0,1.0,89.0,18.0,46.0,1.459277e+09,0.0
1236,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,accepted_at,2016-03-29 18:47:01,3.0,13.0,29.0,1.0,89.0,18.0,47.0,1.459277e+09,11.0
1237,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,arrived_at,2016-03-29 18:50:00,3.0,13.0,29.0,1.0,89.0,18.0,50.0,1.459277e+09,190.0
1238,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,picked_up_at,2016-03-29 18:50:01,3.0,13.0,29.0,1.0,89.0,18.0,50.0,1.459277e+09,191.0
1239,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,dropped_off_at,2016-03-29 18:55:47,3.0,13.0,29.0,1.0,89.0,18.0,55.0,1.459278e+09,537.0


Since part-time drivers should have a lower LTV compared to full-time drivers, we hypothesize that drivers who drive mainly on weekends are part-time drivers.

In [32]:
# Monday 0, Sunday 6
ids_timestamps['is_weekday'] = 0 
ids_timestamps.loc[ids_timestamps['timestampDayofweek'] < 5, 'is_weekday'] = 1

weekday_drivers_g = ids_timestamps.groupby('driver_id')['is_weekday'].mean()

weekday_drivers_g = weekday_drivers_g.reset_index()

weekday_drivers_g.sort_values(by='is_weekday').head()

,driver_id,is_weekday
870,ea71f79a86672486e018deeb47a545ec,0.0
296,4bbf15c7280e29c1df6edd7bf6dfa56a,0.0
108,1cf6fa07dcec364af2acf257b2d3731e,0.0
732,c64e642876910be4a1f0b57404dc9710,0.0
210,364c929f1d6535aa17df83ebcb349e87,0.0


In [33]:
ids_timestamps[ids_timestamps['driver_id'] == 'ea71f79a86672486e018deeb47a545ec']['ride_id'].nunique()

72

Driver `ea71f79a86672486e018deeb47a545ec` made 72 rides in the entire dataset and all rides only happen on weekends.

Next, we look into drivers who prefer to drive in the morning v.s. drivers who prefer to drive at night. We define `is_late_ride` as rides that happen between 11:00 PM and 6:00 AM.

In [34]:
ids_timestamps['is_late_ride'] = 0 
ids_timestamps.loc[(ids_timestamps['timestampHour'] >= 23) |
                   (ids_timestamps['timestampHour'] <= 6), 
                   'is_late_ride'] = 1

is_late_ride_g = ids_timestamps.groupby('driver_id')['is_late_ride'].mean()

is_late_ride_g = is_late_ride_g.reset_index()

is_late_ride_g.sort_values(by='is_late_ride', ascending=False).head()

,driver_id,is_late_ride
523,8969d175c4676642b245af55d2dba19b,0.903448
222,3857da988688981b67c878fc179eb0dc,0.898947
551,905f8007cd46415eba8b9dce088b4395,0.891892
466,793e2072826ee96904a5f03ae2357e96,0.873927
77,175540c364f94b6adbc0ce3541a3254a,0.870370


In [0]:
ids_timestamps[ids_timestamps['driver_id'] == '3857da988688981b67c878fc179eb0dc']['ride_id'].nunique()

190

Driver `3857da988688981b67c878fc179eb0dc` completed 190 rides in the entire dataset and on most of the rides are completed at night.

Next, we create a feature `drop_requested_lag` to measure how fast do drivers pick up the next ride after they drop off passengers. The feature `drop_requested_lag` only considers same day pickups. So there exists a condition that a driver only picks up a passenger a day and this might lead to low `drop_requested_lag` value.

In [40]:
# shift the timestamp down 1 row
# and subtract the timestamp_shift from timestamp
# to measure the lag between a dropoff and request time
# since most request time is relatively close to pickup time
# this can be a good proxy
ids_timestamps['timestamp_shift'] = ids_timestamps.groupby('driver_id')['timestamp'].apply(lambda o: o.shift())

ids_timestamps['delta_timestamp'] = ids_timestamps['timestamp'] - ids_timestamps['timestamp_shift']

ids_timestamps.head(8)

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,fare,event,timestamp,timestampMonth,timestampWeek,timestampDay,timestampDayofweek,timestampDayofyear,timestampHour,timestampMinute,timestampElapsed,pickup_window,is_weekday,is_late_ride,timestamp_shift,delta_timestamp,drop_requested_lag
1235,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,requested_at,2016-03-29 18:46:50,3.0,13.0,29.0,1.0,89.0,18.0,46.0,1.459277e+09,0.0,1,0,NaT,NaT,NaN
1236,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,accepted_at,2016-03-29 18:47:01,3.0,13.0,29.0,1.0,89.0,18.0,47.0,1.459277e+09,11.0,1,0,2016-03-29 18:46:50,00:00:11,NaN
1237,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,arrived_at,2016-03-29 18:50:00,3.0,13.0,29.0,1.0,89.0,18.0,50.0,1.459277e+09,190.0,1,0,2016-03-29 18:47:01,00:02:59,0.0
1238,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,picked_up_at,2016-03-29 18:50:01,3.0,13.0,29.0,1.0,89.0,18.0,50.0,1.459277e+09,191.0,1,0,2016-03-29 18:50:00,00:00:01,0.0
1239,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,dropped_off_at,2016-03-29 18:55:47,3.0,13.0,29.0,1.0,89.0,18.0,55.0,1.459278e+09,537.0,1,0,2016-03-29 18:50:01,00:05:46,0.0
5,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,2.089055,13.483333,NaN,9.118747,requested_at,2016-03-29 19:00:49,3.0,13.0,29.0,1.0,89.0,19.0,0.0,1.459278e+09,0.0,1,0,2016-03-29 18:55:47,00:05:02,419904.0
6,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,2.089055,13.483333,NaN,9.118747,accepted_at,2016-03-29 19:00:52,3.0,13.0,29.0,1.0,89.0,19.0,0.0,1.459278e+09,3.0,1,0,2016-03-29 19:00:49,00:00:03,305.0
7,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,2.089055,13.483333,NaN,9.118747,arrived_at,2016-03-29 19:03:57,3.0,13.0,29.0,1.0,89.0,19.0,3.0,1.459278e+09,188.0,1,0,2016-03-29 19:00:52,00:03:05,0.0


In [0]:
ids_timestamps.loc[(ids_timestamps['event'] == 'requested_at') &
                   (~ids_timestamps['delta_timestamp'].isna()), 'drop_requested_lag'] = 1

ids_timestamps['drop_requested_lag'].fillna(0, inplace=True)

ids_timestamps['drop_requested_lag'] = ids_timestamps['drop_requested_lag'] * ids_timestamps['delta_timestamp']

ids_timestamps['drop_requested_lag'] = ids_timestamps['drop_requested_lag'].apply(lambda o: o.total_seconds())

ids_timestamps.loc[ids_timestamps['delta_timestamp'].dt.days > 0, 'drop_requested_lag'] = np.nan

In [43]:
filtered_requested_at = ids_timestamps[ids_timestamps['event'] == 'requested_at']

drop_requested_lag_g = filtered_requested_at.groupby('driver_id')['drop_requested_lag'].mean()

drop_requested_lag_g = drop_requested_lag_g.reset_index()

drop_requested_lag_g.sort_values(by='drop_requested_lag').head()

,driver_id,drop_requested_lag
441,7ff85c5c0e9324e28d1e0d0589c364bd,181.000000
526,9a54684a69721c1075c2af5fc077665b,754.351351
495,905f8007cd46415eba8b9dce088b4395,936.774194
226,42256e33936dfa69088f540a720edc97,942.870968
313,5cf93f7d1d3a8f0cf395c84053c31b1b,976.818182


Merge `ids_timestamps` with `driver_id`.

In [0]:
df = pd.merge(ids_timestamps, driver_ids,
              left_on='driver_id', right_on='driver_id')

To calculate how many actual days did the driver drive, we calculate the number of unique days the driver has activity.

In [45]:
df['date'] = df['timestamp'].dt.date

unique_dates_g = df.groupby('driver_id')['date'].nunique()

unique_dates_g = unique_dates_g.reset_index()

unique_dates_g.rename(columns={'date': 'unique_days'}, inplace=True)

unique_dates_g.head()

,driver_id,unique_days
0,002be0ffdc997bd5c50703158b7c2491,56
1,007f0389f9c7b03ef97098422f902e62,12
2,011e5c5dfc5c2c92501b8b24d47509bc,12
3,0152a2f305e71d26cc964f8d4411add9,42
4,01674381af7edd264113d4e6ed55ecda,40


Next, we create the feature `driving_period` to account for the duration between the last trip a driver made and the day the driver was onboard. 

In [0]:
df['driving_period'] = (df.groupby('driver_id')['timestamp'].tail(1) - pd.to_datetime(df['driver_onboard_date'])).dt.days

To calculate the average churn rate, we first get the driver's last activity based on `timestamp` in `ride_timestamps`. Since the last day of the `timestamp` in `ride_timestamps` was 2016/06/27, we calculate the number of days since the driver's last activity respect to the last day of the dataset. We assume drivers who have more than 14 days of inactivity have churned.

In [0]:
df['last_activity'] = (df['timestamp'].max() - df.groupby('driver_id')['timestamp'].tail(1)).dt.days

last_activity_df = df.groupby('driver_id').tail(1)

In [48]:
churn_rate = len(last_activity_df[last_activity_df['last_activity'] > 14]) / len(last_activity_df)

churn_rate

0.28805620608899296

In [51]:
onboard_period = df[~df['driving_period'].isna()]

onboard_period = onboard_period[['driver_id', 'driving_period']].copy()

onboard_period.sort_values('driving_period').head()

,driver_id,driving_period
715947,bd057e02f75c92917389d90bb215fe91,1.0
404840,68b546b2f4102641d6774c1ce4f57457,1.0
429430,6eb2e0f41fc9f1fbb70e7751035fdf87,1.0
609573,a2334fdb829cf96fbae920df0cce1587,2.0
841116,e4f3a9d5cf57b9b518136afd9757f76d,3.0


## LTV
We next calculate the LTV for drivers.

In [0]:
ltv = ride_id_g

to_merge = (drop_requested_lag_g, ride_prime_time_g, fare_g, 
            onboard_period, window_g, unique_dates_g,
            weekday_drivers_g, is_late_ride_g)

for ser in to_merge:
    ltv = pd.merge(ltv, ser, on='driver_id', how='outer')

In [59]:
ltv.isna().sum()

driver_id             0
ride_count            0
drop_requested_lag    0
ride_prime_time       0
fare                  0
driving_period        0
pickup_window         0
unique_days           0
is_weekday            0
is_late_ride          0
dtype: int64

It is hard to decide a driver's `drop_requested_lag` if there is he or she does not have any record of ride in `ride_timestamps`. Hence, we decide to impute the `drop_requested_lag`'s `NaN`s as the mean of all `drop_requested_lag`. Same goes for all features that require entries in `ride_timestamps` for calculation.

In [0]:
ltv['driving_period'].fillna(ltv['driving_period'], inplace=True)

ltv['drop_requested_lag'].fillna(ltv['drop_requested_lag'], inplace=True)

ltv['pickup_window'].fillna(ltv['pickup_window'], inplace=True)

ltv['unique_days'].fillna(ltv['unique_days'].mean(), inplace=True)
ltv['unique_days'].replace(0, ltv['unique_days'].mean(), inplace=True)

ltv['drop_requested_lag'].fillna(ltv['drop_requested_lag'].mean(), inplace=True)

ltv['driving_period'].fillna(ltv['driving_period'].mean(), inplace=True)

ltv['pickup_window'].fillna(ltv['pickup_window'].mean(), inplace=True)

In [56]:
ltv.head()

,driver_id,ride_count,drop_requested_lag,ride_prime_time,fare,driving_period,pickup_window,unique_days,is_weekday,is_late_ride
0,002be0ffdc997bd5c50703158b7c2491,277,6594.516129,48.863636,3560.926071,86.0,8.407942,56.0,0.769675,0.238989
1,007f0389f9c7b03ef97098422f902e62,31,4615.550000,52.083333,321.494948,85.0,11.838710,12.0,0.709677,0.000000
2,011e5c5dfc5c2c92501b8b24d47509bc,34,7185.041667,42.187500,482.428390,68.0,7.294118,12.0,0.529412,0.058824
3,0152a2f305e71d26cc964f8d4411add9,191,10028.796610,42.708333,2610.871561,64.0,14.198953,42.0,0.796859,0.459686
4,01674381af7edd264113d4e6ed55ecda,375,6633.404372,47.474747,5381.097539,56.0,12.528000,40.0,1.000000,0.054933


## Questions
* What are the main factors that affect a driver's lifetime value?
* What is the average projected lifetime of a driver? That is, once a driver is
onboarded, how long do they typically continue driving with Lyft?
* Do all drivers act alike? Are there specific segments of drivers that generate more
value for Lyft than the average driver?
* What actionable recommendations are there for the business?

In [0]:
# https://blog.hubspot.com/service/how-to-calculate-customer-lifetime-value
ltv['average_ride_val'] = ltv['fare'] / ltv['ride_count']

ltv['average_ride_freq'] = ltv['ride_count'] / ltv['unique_days']

ltv['average_val'] = ltv['average_ride_val'] * ltv['average_ride_freq']

ltv['ltv'] = ltv['average_val'] * 365 / churn_rate

In [61]:
base_metrics = ['ride_count', 'fare', 'unique_days', 'ltv']

ltv[['driver_id'] + base_metrics].sort_values(by='ltv', ascending=False).head()

,driver_id,ride_count,fare,unique_days,ltv
412,6b65c06851e944351dd285a1eb729499,718,10708.943770,45.0,301543.348024
217,3788dc9e91f1548816ce8b5af07ddadc,783,12304.887351,56.0,278423.086246
156,297e507cc9f1da096e51c2223657255b,588,8657.558363,40.0,274252.796465
318,51b528390e8e7780595f6009b1f4cf72,240,3018.980483,14.0,273242.278251
426,6eb2e0f41fc9f1fbb70e7751035fdf87,35,421.815299,2.0,267243.997669


The main factors that affect lifetime value (LTV) are the number of rides a driver has completed, `ride_count`, total revenue generated by the driver, `fare` and the number of days the driver worked, `unique_days`.

In [65]:
1 / churn_rate

3.4715447154471546

We take average lifetime as the inverse of churn rate. Hence, we predict the average lifetime for drivers is approximately 3.5 years.

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [66]:
all_metrics = [col for col in ltv.columns if col not in ['driver_id']]

all_metrics

['ride_count',
 'drop_requested_lag',
 'ride_prime_time',
 'fare',
 'driving_period',
 'pickup_window',
 'unique_days',
 'is_weekday',
 'is_late_ride',
 'average_ride_val',
 'average_ride_freq',
 'average_val',
 'ltv']

In [67]:
# only fit base_metrics for KMeans
lvt_matrix = ltv[base_metrics].values

CLUSTERS = 3

kmeans = KMeans(n_clusters=CLUSTERS, init='k-means++', n_init=40)
kmeans.fit(lvt_matrix)

clusters = kmeans.predict(lvt_matrix)

print("The average silhouette_score is :", silhouette_score(lvt_matrix, clusters))

The average silhouette_score is : 0.5658765040390985


In [68]:
pd.Series(clusters).value_counts()

1    401
0    375
2    161
dtype: int64

In [69]:
ltv['clusters'] = clusters

scale_max = ltv[all_metrics].max().values

scale_max

array([9.19000000e+02, 3.85809615e+04, 1.46875000e+02, 1.23502881e+04,
       9.00000000e+01, 4.12500000e+01, 8.10000000e+01, 1.00000000e+00,
       9.03448276e-01, 2.59015036e+01, 1.75000000e+01, 2.37976528e+02,
       3.01543348e+05])

In [0]:
import plotly_express as px

In [71]:
for i in range(3):
    radar = pd.DataFrame(ltv[ltv['clusters'] == i][all_metrics].mean()).reset_index()

    radar['r'] = radar.loc[:, 0] / scale_max

    radar.rename(columns={'index': 'theta'}, inplace=True)

    print('Cluster {}'.format(str(i)))
    fig = px.line_polar(radar, r='r', theta='theta', line_close=True)
    fig.show()

Cluster 0


Cluster 1


Cluster 2
